# 📊 ETL e Análise de Dados de Biometria de Tilápias

## 📦 1. Importação de Bibliotecas

In [1]:
import os
import pandas as pd
from glob import glob
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display

## ⚙️ 2. Função de ETL para Leitura dos Arquivos de Biometria

In [2]:
def load_biometria_data(path_folder="biometria"):
    all_files = glob(os.path.join(path_folder, "*.xlsx"))
    all_data = []

    for file in all_files:
        try:
            xls = pd.ExcelFile(file)
            if "01 (2)" not in xls.sheet_names:
                print(f"⚠️ Aba '01 (2)' não encontrada em: {file}")
                continue

            df = xls.parse("01 (2)", header=12)
            df.columns = [str(col).strip().upper().replace(" ", "_") for col in df.columns]
            df["FILENAME"] = os.path.basename(file)

            if "ULTIMA_BIOMETRIA" in df.columns:
                df["DATA_BIOMETRIA"] = pd.to_datetime(df["ULTIMA_BIOMETRIA"], errors="coerce")
            else:
                df["DATA_BIOMETRIA"] = pd.NaT

            cols_to_keep = {
                "UNID.PRODUÇÃO": "TANQUE",
                "PESO_ATUAL": "PESO_ATUAL",
                "Nº_DE_PEIXES_ATUAL": "N_PEIXES_ATUAL",
                "BIOM_ATUAL": "BIOMASSA_ATUAL"
            }

            missing_cols = [col for col in cols_to_keep if col not in df.columns]
            if missing_cols:
                print(f"⚠️ Colunas ausentes em {file}: {missing_cols}")
                continue

            temp_df = df[list(cols_to_keep.keys())].rename(columns=cols_to_keep)
            temp_df["DATA_BIOMETRIA"] = df["DATA_BIOMETRIA"]
            temp_df["FILENAME"] = df["FILENAME"]

            temp_df = temp_df.dropna(subset=["TANQUE", "DATA_BIOMETRIA", "PESO_ATUAL"])
            all_data.append(temp_df)

        except Exception as e:
            print(f"⚠️ Erro ao ler {file}: {e}")

    if all_data:
        return pd.concat(all_data, ignore_index=True)
    else:
        return pd.DataFrame()

## 📈 3. Visualização Interativa do Peso por Biometria

In [3]:
def plot_peso_biometria(df_biom):
    df_biom["DATA_BIOMETRIA"] = pd.to_datetime(df_biom["DATA_BIOMETRIA"], errors="coerce")
    tanques = sorted(df_biom["TANQUE"].dropna().unique())
    dropdown = widgets.Dropdown(options=tanques, description='Tanque:', layout=widgets.Layout(width='50%'))

    def update_plot(tanque):
        df_tanque = df_biom[df_biom["TANQUE"] == tanque].sort_values("DATA_BIOMETRIA")

        fig = px.line(
            df_tanque,
            x="DATA_BIOMETRIA",
            y="PESO_ATUAL",
            title=f"Evolução do Peso Atual - {tanque}",
            markers=True,
            labels={
                "DATA_BIOMETRIA": "Data da Biometria",
                "PESO_ATUAL": "Peso Médio (g)"
            }
        )

        fig.update_traces(line=dict(width=3, color="black"),
                          marker=dict(size=8, color="darkblue", symbol="circle"))

        fig.update_layout(
            template="plotly_white",
            title_font=dict(size=20, color='black'),
            font=dict(color='black'),
            title_x=0.5,
            plot_bgcolor='white',
            paper_bgcolor='white',
            xaxis=dict(
                title="Data",
                showgrid=True,
                gridcolor='lightgray',
                linecolor='black',
                tickfont=dict(color='black'),
                titlefont=dict(color='black'),
                tickformat='%b/%Y'
            ),
            yaxis=dict(
                title="Peso Atual (g)",
                showgrid=True,
                gridcolor='lightgray',
                linecolor='black',
                tickfont=dict(color='black'),
                titlefont=dict(color='black')
            ),
            margin=dict(t=60, l=60, r=40, b=60)
        )

        fig.show()

    widgets.interact(update_plot, tanque=dropdown)

## 📊 4. Análise Estatística da Biometria

In [4]:
# Suponha que df_biometria_final já foi carregado
df_biometria_final["DATA_BIOMETRIA"] = pd.to_datetime(df_biometria_final["DATA_BIOMETRIA"])
df_biometria_final["PESO_ATUAL"] = pd.to_numeric(df_biometria_final["PESO_ATUAL"], errors="coerce")
df_biometria_final["N_PEIXES_ATUAL"] = pd.to_numeric(df_biometria_final["N_PEIXES_ATUAL"], errors="coerce")
df_biometria_final["BIOMASSA_ATUAL"] = pd.to_numeric(df_biometria_final["BIOMASSA_ATUAL"], errors="coerce")

# Estatísticas descritivas
df_biometria_final.describe()

# GMD
df_biometria_final = df_biometria_final.sort_values(["TANQUE", "DATA_BIOMETRIA"])
df_biometria_final["PESO_ANTERIOR"] = df_biometria_final.groupby("TANQUE")["PESO_ATUAL"].shift(1)
df_biometria_final["DATA_ANTERIOR"] = df_biometria_final.groupby("TANQUE")["DATA_BIOMETRIA"].shift(1)
df_biometria_final["DIAS_ENTRE_BIOMETRIAS"] = (df_biometria_final["DATA_BIOMETRIA"] - df_biometria_final["DATA_ANTERIOR"]).dt.days
df_biometria_final["GMD"] = (df_biometria_final["PESO_ATUAL"] - df_biometria_final["PESO_ANTERIOR"]) / df_biometria_final["DIAS_ENTRE_BIOMETRIAS"]

# Ranking por GMD
df_biometria_final.groupby("TANQUE")["GMD"].mean().sort_values(ascending=False)

NameError: name 'df_biometria_final' is not defined